In [1]:
import folium
import osmnx as ox
import pandas as pd
import os
import pickle
from collections import defaultdict
import cv2
import pylab as pl

# Load Cameras (both manual and pred+confirmed)

In [12]:
manual = pd.read_csv('data/manual_cameras_wboag.tsv', sep='\t')
manual['Dome Camera'  ] = manual['Dome Camera'  ].fillna(0)
manual['Button Camera'] = manual['Button Camera'].fillna(0)
manual['Police Box'   ] = manual['Police Box'   ].fillna(0)

display(manual[['Dome Camera','Button Camera', 'Police Box']].sum(axis=0))

manual

Dome Camera      161.0
Button Camera     37.0
Police Box        66.0
dtype: float64

,Latitude,Longitude,Name,Dome Camera,Button Camera,Police Box,Notes,Region,URL
0,42.365432,-71.061052,Causeway and Canal,1.0,0.0,1.0,NaN,West End,"https://www.google.com/maps/@42.3654325,-71.06..."
1,42.363682,-71.058375,New Chardon and John Fitzgerald,1.0,0.0,0.0,NaN,West End,"https://www.google.com/maps/@42.3636822,-71.05..."
2,42.332230,-71.071605,Mass Ave and Gerard,0.0,3.0,1.0,NaN,Roxbury,"https://www.google.com/maps/@42.3322297,-71.07..."
3,42.330624,-71.070058,Mass Ave and Chesterton,0.0,1.0,1.0,NaN,Roxbury,"https://www.google.com/maps/@42.3306241,-71.07..."
4,42.326835,-71.066936,Mass Ave and Newmarket,1.0,0.0,0.0,NaN,Roxbury,"https://www.google.com/maps/@42.3268348,-71.06..."
...,...,...,...,...,...,...,...,...,...
148,42.364300,-71.063446,Staniford and Lomasney,2.0,1.0,1.0,NaN,West End,"https://www.google.com/maps/@42.3642999,-71.06..."
149,42.363862,-71.063149,Staniford and Merrimac,1.0,0.0,0.0,NaN,West End,"https://www.google.com/maps/@42.3638619,-71.06..."
150,42.362982,-71.059962,New Chardon and Merrimac,1.0,0.0,0.0,NaN,West End,"https://www.google.com/maps/@42.3629823,-71.05..."
151,42.352367,-71.073623,Berkeley,1.0,0.0,1.0,NaN,Back Bay,"https://www.google.com/maps/@42.3523674,-71.07..."


In [14]:
pred = pd.read_csv('data/pred_cameras_wboag.tsv', sep='\t')
pred['max_prob'] = pred['probs'].apply(lambda t: float(t[1:].split(',')[0]))
pred['confirm dome'] = pred['confirm dome'].fillna(0)
pred['confirm button'] = pred['confirm button'].fillna(0)
pred['confirm box'] = pred['confirm box'].fillna(0)
pred = pred[(pred['confirm dome'] + pred['confirm button'] + pred['confirm box'])>0]

display(pred[['confirm dome','confirm button', 'confirm box']].sum(axis=0))

pred

confirm dome      158.0
confirm button     39.0
confirm box        92.0
dtype: float64

,File,Latitude,Longitude,probs,confirm dome,confirm button,confirm box,max_prob
0,image4029.jpg,42.317876,-71.081978,"[0.9917173981666565, 0.39172911643981934, 0.31...",0.0,1.0,1.0,0.991717
1,image20136.jpg,42.328508,-71.079257,"[0.9908430576324463, 0.39918622374534607, 0.22...",1.0,0.0,1.0,0.990843
2,image4546.jpg,42.355705,-71.062844,"[0.988140344619751, 0.6381937861442566, 0.4311...",1.0,0.0,1.0,0.988140
3,image34545.jpg,42.305539,-71.067866,"[0.9846373200416565, 0.4426528513431549, 0.433...",1.0,0.0,1.0,0.984637
4,image12232.jpg,42.341331,-71.076554,"[0.9824907183647156, 0.5959391593933105, 0.338...",1.0,1.0,1.0,0.982491
...,...,...,...,...,...,...,...,...
291,image6464.jpg,42.311607,-71.070919,"[0.6956035494804382, 0.5542750358581543, 0.238...",1.0,0.0,1.0,0.695604
293,image32232.jpg,42.270055,-71.093357,"[0.694869339466095, 0.35613206028938293, 0.310...",1.0,0.0,0.0,0.694869
298,image17425.jpg,42.385425,-71.010515,"[0.6907843947410583, 0.43976086378097534, 0.35...",1.0,0.0,0.0,0.690784
299,image13705.jpg,42.329651,-71.074960,"[0.6895087361335754, 0.2393171787261963, 0.158...",0.0,0.0,1.0,0.689509


# Non-Dedeuplicated Display

In [4]:
# Create a map
b_folium = folium.Map(location=[42.3351457,-71.0748679], tiles='openstreetmap', zoom_start=13)

#add red marker for each manual camera
for i,(_,row) in enumerate(manual.iterrows()):
    icon = folium.map.Icon(color='blue')
    coord = (row.Latitude, row.Longitude)
    folium.Marker(coord, popup=f'(coord)', tooltip=i, icon=icon).add_to(b_folium)
    
#add blue marker for each predicted camera
for i,(_,row) in enumerate(pred.iterrows()):
    icon = folium.map.Icon(color='red')
    coord = (row.Latitude, row.Longitude)
    folium.Marker(coord, popup=f'(coord)', tooltip=i, icon=icon).add_to(b_folium)

# display folium
b_folium

# Simple Overlap Detection

Assume that no predictions double-counted the same camera (slightly naive) and count reds + blues + purples

In [5]:
def euclidian(a, b):
    c2 = (a[0]-b[0])**2 + (a[1]-b[1])**2
    return c2 ** 0.5

In [6]:
pred_coords = set()
for i,row in enumerate(pred.itertuples()):
    coord = (row.Latitude, row.Longitude)
    pred_coords.add(coord)
    
manual_coords = set()
for i,row in enumerate(manual.itertuples()):
    coord = (row.Latitude, row.Longitude)
    manual_coords.add(coord)

In [11]:
# Create a map
b_folium = folium.Map(location=[42.321207,-71.077061], tiles='openstreetmap', zoom_start=14)

num_dome = 0
num_button = 0
num_box = 0

# Manually Identified
for i,(_,row) in enumerate(manual.iterrows()):
    coord = (row.Latitude, row.Longitude)

    # Check if predicted manual (ie very very close)
    pred_match = False
    for c in pred_coords:
        if euclidian(coord, c) < .0003:
            pred_match = True
            
    if pred_match:
        # Will be printed below
        continue
    else:
        icon = folium.map.Icon(color='blue')
        folium.Marker(coord, popup=f'{coord}', tooltip=str(coord), icon=icon).add_to(b_folium)
    
        num_dome   += row['Dome Camera']
        num_button += row['Button Camera']
        num_box    += row['Police Box']
    

for i,(_,row) in enumerate(pred.iterrows()):
    coord = (row.Latitude, row.Longitude)
    
    # Check if matches manual (ie very very close)
    manual_match = False
    for c in manual_coords:
        if euclidian(coord, c) < .0003:
            manual_match = True

    if manual_match:
        color = 'purple'
    else:
        color = 'red'

    # plot the TP or FP
    icon = folium.map.Icon(color=color)
    folium.Marker(coord, popup=f'{coord}', tooltip=str(coord), icon=icon).add_to(b_folium)

    num_dome   += row['confirm dome']
    num_button += row['confirm button']
    num_box    += row['confirm box']
    
    
print('Dome:  ', num_dome)
print('Button:', num_button)
print('Box:   ', num_box)
print()

b_folium  

Dome:   269.0
Button: 59.0
Box:    126.0

